### Convert raw data to BIDS format
Adapted from John Veillette, see the original script [here](https://github.com/john-veillette/eeg-training-materials/blob/main/convert-to-bids.ipynb). I removed all the comments and explanations that were in the original notebook because only keeping a portion of them seems more confusing and irresponsible than redirecting anyone interested in how this script actually works to the original file with Johns exhaustive annotations.

In [1]:
from mne_bids import BIDSPath, write_raw_bids, get_anonymization_daysback
import pandas as pd
import numpy as np
import itertools
import mne
import os
import re

In [2]:
# Constants
DATA_DIR = '../data/raw/' # where our data currently lives
BIDS_DIR = '../data/bids/' # where we want it to live
MAPS_DIR = '../data/captrak/' # where the mapping and electrode location files love

#### Parse filenames

In [3]:
# Get filenames and digest them
fnames = os.listdir(DATA_DIR)
fnames = [f for f in fnames if '.vhdr' in f] # filter for .vhdr files

# Get subject list from file order
filter_subs = re.compile('letty_subj_(\w?).*') # create regex filter
subs = list(map(filter_subs.findall, fnames)) # extract subject numbers with filter
subs = list(itertools.chain(*subs)) # flatten then nested list

# Get a task list
tasks = ['pitch']*len(subs) # broadcast the only task name

# Get a run list
filter_runs = re.compile('\w+[0-9]_([0-9]).*')
runs = list(map(filter_runs.findall, fnames))
runs = ['1' if x == [] else x for x in runs]
runs = list(itertools.chain(*runs))

#### Retrieve mappings between channel numbers and channel names

In [4]:
# For subj 2, 3, 5, 6
mapping_table = pd.read_csv(MAPS_DIR + 'pitch_tracking_64_at_IZ.csv')
mapping_64_at_IZ = {mapping_table.number[i]: mapping_table.name[i] for i in range(len(mapping_table))}

# For subj 4 IZ is excluded but channel 64 is not moved to FCZ
mapping_table = pd.read_csv(MAPS_DIR + 'pitch_tracking_no_IZ.csv')
mapping_no_IZ = {mapping_table.number[i]: mapping_table.name[i] for i in range(len(mapping_table))}

# For subj 7, and onwards
mapping_table = pd.read_csv(MAPS_DIR + 'pitch_tracking_64_at_FCZ.csv')
mapping_64_at_FCZ = {mapping_table.number[i]: mapping_table.name[i] for i in range(len(mapping_table))}

# Create dict for subjects and their mappings
mappings = {'2': mapping_64_at_IZ,
           '3': mapping_64_at_IZ, 
           '4': mapping_no_IZ,
           '5': mapping_64_at_IZ,
           '6': mapping_64_at_IZ,
           '7': mapping_64_at_FCZ}

#### Run conversion on all files

In [6]:
for i in range(len(fnames)):
    sub = subs[i]
    task = tasks[i]
    run = runs[i]
    fpath = os.path.join(DATA_DIR, fnames[i])
    print(fpath)

    # load data with MNE function for your file format
    raw = mne.io.read_raw_brainvision(fpath)
    raw.load_data()
    raw.set_channel_types({'Aux1': 'stim'})

    # add some info BIDS will want
    raw.info['line_freq'] = 60 # the power line frequency in the building we collected in

    # map channel numbers to channel names
    mapping = mappings[sub]
    raw.rename_channels(mapping)
    raw.add_reference_channels(ref_channels = ['Cz'])

    # map channels to their coordinates
    dig = mne.channels.read_dig_captrak(MAPS_DIR + 'subj_' + sub + '.bvct')
    raw = mne.set_bipolar_reference(raw, anode = 'Fp1', cathode = 'AF7', ch_name = 'eog1', drop_refs = False)
    raw = mne.set_bipolar_reference(raw, anode = 'Fp2', cathode = 'AF8', ch_name = 'eog2', drop_refs = False)
    raw.set_channel_types({'eog1': 'eog', 'eog2': 'eog', 'AF7': 'eog', 'AF8': 'eog'})
    raw.set_montage(dig, on_missing = 'warn') if sub == '4' else raw.set_montage(dig) 

    # # drop meaningless event name
    events, event_ids = mne.events_from_annotations(raw)
    events = events[events[:,2] != event_ids['New Segment/'], :]

    # # rename events to their stimulus pitch
    event_codes = events[:,2]
    baseline_code = np.argmax(np.bincount(event_codes)) # the one with more trials
    event_names = {1: '50', 2: '100', 3: '150', 4: '200', 5: '250'}
    annot = mne.annotations_from_events(events, sfreq = raw.info['sfreq'], event_desc = event_names)
    raw.set_annotations(annot)

    # build appropriate BIDS directory structure 
    bids_path = BIDSPath(
        run = run,
        subject = sub, 
        task = task, 
        datatype = 'eeg', 
        root = BIDS_DIR
    )

    # get range of dates the BIDS specfiication will accept
    daysback_min, daysback_max = get_anonymization_daysback(raw)

    # write data into BIDS directory, while anonymizing
    write_raw_bids(
        raw, 
        bids_path = bids_path, 
        allow_preload = True, # whether to load full dataset into memory when copying
        format = 'BrainVision', # format to save to
        anonymize = dict(daysback = daysback_min), # shift dates by daysback
        overwrite = True,
    )

../data/raw/letty_subj_3_2.vhdr
Extracting parameters from ../data/raw/letty_subj_3_2.vhdr...
Setting channel info structure...
Reading 0 ... 4713499  =      0.000 ...   942.700 secs...


/var/folders/hv/vj9_d69170l26hyj9hqqj45h0000gn/T/ipykernel_46195/4032613005.py:11: RuntimeWarning: The unit for channel(s) Aux1 has changed from V to NA.
  raw.set_channel_types({'Aux1': 'stim'})


Location for this channel is unknown; consider calling set_montage() again if needed.
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=4713500
    Range : 0 ... 4713499 =      0.000 ...   942.700 secs
Ready.
Added the following bipolar channels:
eog1
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=4713500
    Range : 0 ... 4713499 =      0.000 ...   942.700 secs
Ready.
Added the following bipolar channels:
eog2
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5']
Writing '../data/bids/participants.tsv'...
Writing '../data/bids/participants.json'...
Writing electrodes file to... ../data/bids/sub-3/eeg/sub-3_electrodes.tsv
Writing coordsytem file to... ../data/bids/sub-3/eeg/sub-3_coordsystem.json
Writing '../data/bids/sub-3/eeg/sub-3_electrodes.tsv'...
Writing '../data/bids/sub-3/eeg/sub-3_coordsyst

/var/folders/hv/vj9_d69170l26hyj9hqqj45h0000gn/T/ipykernel_46195/4032613005.py:52: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):
/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):
/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/pybv/io.py:394: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


Writing '../data/bids/sub-3/sub-3_scans.tsv'...
Wrote ../data/bids/sub-3/sub-3_scans.tsv entry with eeg/sub-3_task-pitch_run-2_eeg.vhdr.
../data/raw/letty_subj_6.vhdr
Extracting parameters from ../data/raw/letty_subj_6.vhdr...
Setting channel info structure...
Reading 0 ... 9212749  =      0.000 ...  1842.550 secs...


...

Writing '../data/bids/sub-2/sub-2_scans.tsv'...
Wrote ../data/bids/sub-2/sub-2_scans.tsv entry with eeg/sub-2_task-pitch_run-1_eeg.vhdr.
../data/raw/letty_subj_3.vhdr
Extracting parameters from ../data/raw/letty_subj_3.vhdr...
Setting channel info structure...
Reading 0 ... 5026249  =      0.000 ...  1005.250 secs...


/var/folders/hv/vj9_d69170l26hyj9hqqj45h0000gn/T/ipykernel_46195/4032613005.py:11: RuntimeWarning: The unit for channel(s) Aux1 has changed from V to NA.
  raw.set_channel_types({'Aux1': 'stim'})


Location for this channel is unknown; consider calling set_montage() again if needed.
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=5026250
    Range : 0 ... 5026249 =      0.000 ...  1005.250 secs
Ready.
Added the following bipolar channels:
eog1
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=5026250
    Range : 0 ... 5026249 =      0.000 ...  1005.250 secs
Ready.
Added the following bipolar channels:
eog2
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5']
Writing '../data/bids/participants.tsv'...
Writing '../data/bids/participants.json'...
Writing electrodes file to... ../data/bids/sub-3/eeg/sub-3_electrodes.tsv
Writing coordsytem file to... ../data/bids/sub-3/eeg/sub-3_coordsystem.json
Writing '../data/bids/sub-3/eeg/sub-3_electrodes.tsv'...
Writing '../data/bids/sub-3/eeg/sub-3_coordsyst

/var/folders/hv/vj9_d69170l26hyj9hqqj45h0000gn/T/ipykernel_46195/4032613005.py:52: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):
/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):


Writing '../data/bids/sub-3/sub-3_scans.tsv'...
Wrote ../data/bids/sub-3/sub-3_scans.tsv entry with eeg/sub-3_task-pitch_run-1_eeg.vhdr.


#### Check output files

In [7]:
from mne_bids import print_dir_tree
print_dir_tree(BIDS_DIR)

|/
|--- .DS_Store
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|derivatives/
|--- .DS_Store
|--- preprocessing/
|------ .DS_Store
|------ dataset_description.json
|------ sub-1.html
|------ sub-1/
|--------- sub-1_task-pitches_desc-clean_epo.fif.gz
|sub-1/
|--- eeg/
|sub-2/
|--- .DS_Store
|--- sub-2_scans.tsv
|--- eeg/
|------ sub-2_coordsystem.json
|------ sub-2_electrodes.tsv
|------ sub-2_task-pitch_run-1_channels.tsv
|------ sub-2_task-pitch_run-1_eeg.eeg
|------ sub-2_task-pitch_run-1_eeg.json
|------ sub-2_task-pitch_run-1_eeg.vhdr
|------ sub-2_task-pitch_run-1_eeg.vmrk
|------ sub-2_task-pitch_run-1_events.tsv
|sub-3/
|--- .DS_Store
|--- sub-3_scans.tsv
|--- eeg/
|------ .DS_Store
|------ sub-3_coordsystem.json
|------ sub-3_electrodes.tsv
|------ sub-3_task-pitch_run-1_channels.tsv
|------ sub-3_task-pitch_run-1_eeg.eeg
|------ sub-3_task-pitch_run-1_eeg.json
|------ sub-3_task-pitch_run-1_eeg.vhdr
|------ sub-3_task-pitch_run-1_eeg.vm